In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
#%matplotlib inline
from bs4 import BeautifulSoup
import html5lib
import requests

In [19]:
path = 'http://ergast.com/api/f1'
season = '2016'
race = '2'
URL = path + '/' + season + '/' + race + '/' + 'laps?limit=2500'
results = requests.get(URL)
soup = BeautifulSoup(results.content, 'lxml')

In [20]:
#pull in driverids to names
driver_info = pd.DataFrame.from_csv('driver_info.csv')
driverid_dict = {driver_info.ix[i]['driverid']:driver_info.ix[i]['GivenName'] + " " + driver_info.ix[i]['FamilyName'] for i in driver_info.index}
driverid_dict['sainz'] = driverid_dict['sainz'] + ' Jr.'
driverid_dict

{'alonso': 'Fernando Alonso',
 'bottas': 'Valtteri Bottas',
 'button': 'Jenson Button',
 'ericsson': 'Marcus Ericsson',
 'grosjean': 'Romain Grosjean',
 'gutierrez': 'Esteban Gutiérrez',
 'hamilton': 'Lewis Hamilton',
 'haryanto': 'Rio Haryanto',
 'hulkenberg': 'Nico Hülkenberg',
 'jolyon_palmer': 'Jolyon Palmer',
 'kevin_magnussen': 'Kevin Magnussen',
 'kvyat': 'Daniil Kvyat',
 'massa': 'Felipe Massa',
 'max_verstappen': 'Max Verstappen',
 'nasr': 'Felipe Nasr',
 'perez': 'Sergio Pérez',
 'raikkonen': 'Kimi Räikkönen',
 'ricciardo': 'Daniel Ricciardo',
 'rosberg': 'Nico Rosberg',
 'sainz': 'Carlos Sainz Jr.',
 'vandoorne': 'Stoffel Vandoorne',
 'vettel': 'Sebastian Vettel',
 'wehrlein': 'Pascal Wehrlein'}

In [21]:
headers = ('driverid', 'lap', 'time')
num_rows = len(soup.find_all('timing'))
race_dict = {}
for header in headers:
    race_dict[header] = [race[header] for race in soup.find_all('timing')]
df = pd.DataFrame(race_dict)
df['minutes'] = [x.split(":")[0] for x in df['time']]
df['seconds'] = [x.split(":")[1] for x in df['time']]
df['lap_sec'] = df['minutes'].astype('int64') * 60 + df['seconds'].astype('float32')
df['lap'] = df['lap'].astype('int64')
df['driver'] = [driverid_dict[driver] for driver in df['driverid']]
df

,driverid,lap,time,minutes,seconds,lap_sec,driver
0,rosberg,1,1:39.458,1,39.458,99.458000,Nico Rosberg
1,massa,1,1:41.133,1,41.133,101.132999,Felipe Massa
2,bottas,1,1:41.950,1,41.950,101.950001,Valtteri Bottas
3,ricciardo,1,1:42.560,1,42.560,102.560001,Daniel Ricciardo
4,raikkonen,1,1:43.779,1,43.779,103.778999,Kimi Räikkönen
5,grosjean,1,1:44.134,1,44.134,104.133999,Romain Grosjean
6,hamilton,1,1:44.787,1,44.787,104.786999,Lewis Hamilton
7,gutierrez,1,1:45.365,1,45.365,105.365002,Esteban Gutiérrez
8,max_verstappen,1,1:46.223,1,46.223,106.223000,Max Verstappen
9,button,1,1:46.579,1,46.579,106.578999,Jenson Button


In [22]:
df['lap_sec'].hist()

In [23]:
pivot = df.pivot(index='driver', columns='lap', values='lap_sec')
pivot

lap,1,2,3,4,5,6,7,8,9,10,...,48,49,50,51,52,53,54,55,56,57
driver,,,,,,,,,,,,,,,,,,,,,
Carlos Sainz Jr.,107.730999,133.727000,124.378000,99.525002,99.827999,99.904999,100.007999,100.931999,101.214001,101.370998,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Daniel Ricciardo,102.560001,98.452000,99.231998,99.311001,99.514000,103.243000,117.902000,98.966999,99.258999,99.444000,...,96.573002,96.574001,96.275002,96.444000,96.646000,96.959000,96.426998,96.775002,97.226002,96.705002
Daniil Kvyat,108.782001,101.936001,99.166000,99.594002,99.849998,99.972000,100.553001,100.520000,101.085999,100.341000,...,96.391998,97.625000,98.223999,98.728001,96.726002,96.830002,96.965000,97.735001,98.445000,NaN
Esteban Gutiérrez,105.365002,99.341000,99.341999,99.835999,99.813000,99.792999,99.960999,108.258999,141.635000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Felipe Massa,101.132999,98.217999,99.150002,99.273998,99.415001,99.623001,102.431000,118.988998,99.667999,99.301998,...,98.568001,98.585999,98.761002,99.601002,99.167000,99.410000,99.306000,100.455002,101.367001,NaN
Felipe Nasr,110.174999,101.952000,100.321999,100.541000,100.696999,100.770000,100.754002,100.831001,100.898998,105.445999,...,117.544998,95.360001,96.429001,96.224998,97.202000,97.478001,96.963001,97.084999,97.549999,NaN
Jenson Button,106.578999,99.522999,99.426998,99.750999,99.895000,100.259998,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Kevin Magnussen,110.313000,102.594002,101.069000,100.808998,100.707001,100.964001,101.064999,102.555000,101.278000,101.226002,...,100.203999,100.925999,98.362000,98.113998,98.081001,98.118000,98.612000,98.769001,98.856998,NaN
Kimi Räikkönen,103.778999,98.078999,98.933998,99.062000,99.727001,99.146000,99.113998,98.994999,98.646999,98.980999,...,96.314999,96.166000,96.620998,96.388000,96.550999,96.955002,97.519001,97.682999,97.044998,101.618999


In [24]:
for i in range(2, df['lap'].max() + 1):
    pivot.ix[:,i] = pivot.ix[:,i] + pivot.ix[:, i-1]
pivot

lap,1,2,3,4,5,6,7,8,9,10,...,48,49,50,51,52,53,54,55,56,57
driver,,,,,,,,,,,,,,,,,,,,,
Carlos Sainz Jr.,107.730999,241.457999,365.835999,465.361001,565.189000,665.093998,765.101998,866.033997,967.247998,1068.618996,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Daniel Ricciardo,102.560001,201.012001,300.243999,399.555000,499.069000,602.312000,720.214001,819.181000,918.439999,1017.883999,...,4807.158989,4903.732990,5000.007992,5096.451992,5193.097992,5290.056992,5386.483990,5483.258991,5580.484993,5677.189995
Daniil Kvyat,108.782001,210.718002,309.884003,409.478004,509.328003,609.300003,709.853004,810.373005,911.459003,1011.800003,...,4834.552002,4932.177002,5030.401001,5129.129002,5225.855003,5322.685005,5419.650005,5517.385006,5615.830006,NaN
Esteban Gutiérrez,105.365002,204.706001,304.048000,403.883999,503.696999,603.489998,703.450996,811.709995,953.344995,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Felipe Massa,101.132999,199.350998,298.500999,397.774998,497.189999,596.813000,699.243999,818.232998,917.900997,1017.202995,...,4821.632004,4920.218002,5018.979004,5118.580006,5217.747005,5317.157005,5416.463005,5516.918007,5618.285007,NaN
Felipe Nasr,110.174999,212.126999,312.448997,412.989998,513.686996,614.456997,715.210999,816.042000,916.940998,1022.386997,...,4888.072994,4983.432994,5079.861995,5176.086994,5273.288993,5370.766994,5467.729995,5564.814994,5662.364994,NaN
Jenson Button,106.578999,206.101997,305.528996,405.279995,505.174995,605.434994,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Kevin Magnussen,110.313000,212.907001,313.976002,414.785000,515.492001,616.456001,717.521000,820.076000,921.354000,1022.580002,...,4856.145002,4957.071000,5055.433000,5153.546998,5251.628000,5349.746000,5448.357999,5547.127000,5645.983999,NaN
Kimi Räikkönen,103.778999,201.857998,300.791996,399.853996,499.580997,598.726997,697.840996,796.835995,895.482994,994.463993,...,4748.430996,4844.596996,4941.217995,5037.605995,5134.156994,5231.111996,5328.630997,5426.313995,5523.358994,5624.977993


In [25]:
unpivot = pivot.stack()
new_df = unpivot.reset_index()
new_df.columns = ['driverid', 'lap', 'cume_sec']
new_df.sort_values('cume_sec', ascending=True, inplace=True)
new_df = new_df.reset_index()
new_df.drop('index', axis=1, inplace=True)
new_df

,driverid,lap,cume_sec
0,Nico Rosberg,1,99.458000
1,Felipe Massa,1,101.132999
2,Valtteri Bottas,1,101.950001
3,Daniel Ricciardo,1,102.560001
4,Kimi Räikkönen,1,103.778999
5,Romain Grosjean,1,104.133999
6,Lewis Hamilton,1,104.786999
7,Esteban Gutiérrez,1,105.365002
8,Max Verstappen,1,106.223000
9,Jenson Button,1,106.578999


In [2]:
from f1 import Team
from f1 import Race

In [26]:
#Set up a Race object, score it, return a dataframe of all points
ThisRace = Race('Bahrain')
df_score = ThisRace.score()
df_score

C:\users\jason\documents\gitrepos\fantasy-formula-one\f1.py:227: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  fantasy_points['Movement_Pts'][fantasy_points['Movement_Pts'] < 0] = 0


,Team,Qualifying,Grid,Eff_Grid,Finish,Qual_Pts,Fin_Pts,Team_Pts,Movement_Pts,Completion,Fst_Lap,Total_Race_Pts
Kimi Räikkönen,Ferrari,4,4,3,2,2,9,1,1,3,0,16
Sebastian Vettel,Ferrari,3,DNS,NaN,DNS,2,0,0,0,0,0,2
Nico Hülkenberg,Force India-Mercedes,8,8,7,15,2,0,1,0,3,0,6
Sergio Pérez,Force India-Mercedes,18,18,14,16,0,0,0,0,3,0,3
Romain Grosjean,Haas-Ferrari,9,9,8,5,1,6,1,3,3,0,14
Esteban Gutiérrez,Haas-Ferrari,13,13,NaN,Ret,1,0,0,0,0,0,1
Pascal Wehrlein,MRT-Mercedes,16,16,12,13,0,0,1,0,3,0,4
Rio Haryanto,MRT-Mercedes,21,20,15,17,0,0,0,0,3,0,3
Stoffel Vandoorne,McLaren-Honda,12,12,10,10,1,1,1,0,3,0,6
Jenson Button,McLaren-Honda,14,14,NaN,Ret,1,0,0,0,0,0,1


In [54]:
#Set up current dataframe
current = pd.DataFrame(columns=['position', 'lap'])
current['position'] = df_score['Grid'].copy()
current['lap'] = 0
current['position'][current['position'] == 'DNS'] = np.NaN
current['position'][current['position'] == 'PL'] = np.NaN
current.sort_values('position', inplace=True)
current

C:\Users\Jason\Anaconda3\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Jason\Anaconda3\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,position,lap
Lewis Hamilton,1,0
Nico Rosberg,2,0
Kimi Räikkönen,4,0
Daniel Ricciardo,5,0
Valtteri Bottas,6,0
Felipe Massa,7,0
Nico Hülkenberg,8,0
Romain Grosjean,9,0
Max Verstappen,10,0
Carlos Sainz Jr.,11,0


In [28]:
#Set all laps completed to zero
ThisRace.laps_completed = {driver:current.ix[driver]['lap'] for driver in current.index}
ThisRace.laps_completed

{'Carlos Sainz Jr.': 0,
 'Daniel Ricciardo': 0,
 'Daniil Kvyat': 0,
 'Esteban Gutiérrez': 0,
 'Felipe Massa': 0,
 'Felipe Nasr': 0,
 'Jenson Button': 0,
 'Jolyon Palmer': 0,
 'Kevin Magnussen': 0,
 'Kimi Räikkönen': 0,
 'Lewis Hamilton': 0,
 'Marcus Ericsson': 0,
 'Max Verstappen': 0,
 'Nico Hülkenberg': 0,
 'Nico Rosberg': 0,
 'Pascal Wehrlein': 0,
 'Rio Haryanto': 0,
 'Romain Grosjean': 0,
 'Sebastian Vettel': 0,
 'Sergio Pérez': 0,
 'Stoffel Vandoorne': 0,
 'Valtteri Bottas': 0}

In [3]:
import datetime
import time

In [55]:
fastest_lap_time = 1000
start_time = datetime.datetime.now()
#whole race
#end_time = start_time + datetime.timedelta(seconds=float(new_df[-1:]['cume_sec']))
#2 min test
end_time = start_time + datetime.timedelta(seconds=120)
print(start_time.isoformat())
print(end_time.isoformat())
i = 0
#Initialize graph
drivers = [driver for driver in current.index]
x_pos = np.arange(len(current))
points = [ThisRace.fantasy_points[driver] for driver in drivers]
plt.ion()
fig = plt.figure()
ax = fig.add_subplot(1,1,1)
plt.bar(x_pos, points, align='center', alpha=0.7)
plt.xticks(x_pos, drivers, rotation=90)
plt.xlim([-1, len(points)])
plt.title('Drivers Fantasy Points')
plt.xlabel('Driver')
plt.ylabel('Points')
plt.ylim([0, 25])
plt.draw()
for i in range(300):
#while datetime.datetime.now() < end_time:
    #if datetime.datetime.now() >= start_time + datetime.timedelta(seconds=new_df.ix[i, 'cume_sec']):
    if i <= 300:
        if new_df.ix[i, 'cume_sec'] != np.NaN:
            #print(new_df.ix[i, 'driverid'], 'crossed the line! ', new_df.ix[i, 'cume_sec'])
            cur_driver = new_df.ix[i, 'driverid']
            cur_lap = new_df.ix[i, 'lap']
            current.ix[cur_driver, 'lap'] += 1 #increment lap
            current.sort_values('lap', kind='mergesort', inplace=True, ascending=False) #sort by lap, stable
            current['position'] = np.arange(1,len(current) + 1) #recalc positions
            ThisRace.laps_completed[cur_driver] += 1
            ThisRace.drivers_position = {driver:current.ix[driver, 'position'] for driver in current.index}
            #print(type(cur_lap), type(new_df.ix[i, 'cume_sec']), type(np.NaN))
            if cur_lap == 1:
                if new_df.ix[i, 'cume_sec'] != np.NaN:
                    cur_lap_time = new_df.ix[i, 'cume_sec']
                    print('Current lap time:', cur_lap_time, cur_driver)
            elif new_df.ix[i, 'cume_sec'] != np.NaN:
                #print(new_df.ix[i, 'cume_sec'])
                #print(float(new_df['cume_sec'][(new_df['driverid'] == cur_driver) & (new_df['lap'] == cur_lap - 1)]))
                cur_lap_time = new_df.ix[i, 'cume_sec'] - float(new_df['cume_sec'][(new_df['driverid'] == cur_driver) & (new_df['lap'] == cur_lap - 1)])
                print('Current lap time:', cur_lap_time, cur_driver)

            #print(new_df['cume_sec'][(new_df['driverid'] == cur_driver) & (new_df['lap'] == 5)])
            if cur_lap_time < fastest_lap_time:
                ThisRace.fastest_lap = cur_driver
                fastest_lap_time = cur_lap_time
                print(cur_driver, 'got the fastest lap!')
            df_score = ThisRace.score()
            df_score.sort_values('Finish', inplace=True, ascending=True)
            #update graph
            drivers = [driver for driver in current.index]
            x_pos = np.arange(len(current))
            points = [ThisRace.fantasy_points[driver] for driver in drivers]
            ax.clear()
            plt.bar(x_pos, points, align='center', alpha=0.7)
            plt.xticks(x_pos, drivers, rotation=90)
            plt.pause(.05)
        i += 1
df_score['Total_Race_Pts'].plot(kind='bar')
current
        

2016-04-18T00:30:24.854472
2016-04-18T00:32:24.854472
Current lap time:

C:\users\jason\documents\gitrepos\fantasy-formula-one\f1.py:227: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  fantasy_points['Movement_Pts'][fantasy_points['Movement_Pts'] < 0] = 0
C:\Users\Jason\Anaconda3\lib\site-packages\matplotlib\backend_bases.py:2437: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


 99.4580001831 Nico Rosberg
Nico Rosberg got the fastest lap!
Current lap time: 101.13299942 Felipe Massa
Current lap time: 101.950000763 Valtteri Bottas
Current lap time: 102.560001373 Daniel Ricciardo
Current lap time: 103.778999329 Kimi Räikkönen
Current lap time: 104.133998871 Romain Grosjean
Current lap time: 104.786998749 Lewis Hamilton
Current lap time: 105.365001678 Esteban Gutiérrez
Current lap time: 106.222999573 Max Verstappen
Current lap time: 106.578998566 Jenson Button
Current lap time: 107.730998993 Carlos Sainz Jr.
Current lap time: 108.005001068 Sergio Pérez
Current lap time: 108.008998871 Stoffel Vandoorne
Current lap time: 108.540000916 Pascal Wehrlein
Current lap time: 108.782001495 Daniil Kvyat
Current lap time: 108.980998993 Nico Hülkenberg
Current lap time: 109.664001465 Marcus Ericsson
Current lap time: 110.037998199 Rio Haryanto
Current lap time: 110.174999237 Felipe Nasr
Current lap time: 110.312999725 Kevin Magnussen
Current lap time: 98.1489982605 Nico Rosbe

,position,lap
Nico Rosberg,1,16
Kimi Räikkönen,2,16
Daniel Ricciardo,3,16
Lewis Hamilton,4,16
Felipe Massa,5,16
Romain Grosjean,6,16
Max Verstappen,7,16
Daniil Kvyat,8,16
Stoffel Vandoorne,9,16
Valtteri Bottas,10,16


In [42]:
fig = plt.figure()
ax = fig.add_subplot(1,1,1)
plt.bar(x_pos, points, align='center', alpha=0.7)
plt.xticks(x_pos, drivers, rotation=90)
plt.xlim([-1, len(points)])
plt.title('Drivers Fantasy Points')
plt.xlabel('Driver')
plt.ylabel('Points')
plt.ylim([0, 25])
plt.draw()

In [43]:
ax.clear()

In [40]:
fig=plt.figure()
plt.axis([0,10,0,1])
plt.ion()
i=0
x=list()
y=list()
while i <10:
    temp_y=np.random.random()
    x.append(i)
    y.append(temp_y)
    plt.scatter(i,temp_y)
    i+=1
    plt.pause(0.5)
    plt.clear()
plt.draw()


C:\Users\Jason\Anaconda3\lib\site-packages\matplotlib\backend_bases.py:2437: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


AttributeError: module 'matplotlib.pyplot' has no attribute 'ax'